In [514]:
import re
import csv

BASE_FILMS_URL = 'base_films_500.csv'

def build_lexicon(csv_file):
    lexicon = {}
    with open(csv_file, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            for col, val in row.items():
                if not val:
                    continue
                col_name = 'actor' if col.lower().startswith('acteur') else col.lower()
                lexicon.setdefault(col_name, set()).add(val.strip())
        return lexicon
lexicon = build_lexicon(BASE_FILMS_URL)

In [515]:
def process_query(query):
    concepts = []
    processed_query = query

    # Find known lexicon values
    for col_name, values in lexicon.items():
        for value in values:
            if value.lower() in processed_query.lower():
                concepts.append({'concept': col_name, 'value': value})
                if col_name == 'genre':
                    processed_query = re.sub(re.escape(value), '<'+col_name+'>', processed_query, flags=re.IGNORECASE)
                else:
                    processed_query = re.sub(r'\b' + re.escape(value) + r'\b', '<'+col_name+'>', processed_query)

    # Capture years before replacing
    found_years = re.findall(r'\b\d{4}\b', processed_query)
    for year in found_years:
        concepts.append({'concept': 'annee', 'value': year})
    processed_query = re.sub(r'\b\d{4}\b', '<annee>', processed_query)

    # Capture the text following "titre LIKE" (ignoring case) and add to concepts
    titre_matches = re.finditer(
        r"(?i)(titre\s+LIKE\s+')([\%\_]*)([^'\%\_]*)([\%\_]*)(')",
        processed_query
    )
    for match in titre_matches:
        # Group 3 is the text that will be replaced by <titre>
        replaced_text = match.group(3).strip()
        if replaced_text:
            concepts.append({'concept': 'titre', 'value': replaced_text})

    # Now do the actual replacement preserving % or _
    processed_query = re.sub(
        r"(?i)(titre\s+LIKE\s+')([\%\_]*)([^'\%\_]*)([\%\_]*)(')",
        r"\1\2<titre>\4\5",
        processed_query
    )

    return processed_query, concepts

In [516]:
query_test_Like = "SELECT * FROM films WHERE titre LIKE '%Promise' ORDER BY annee DESC;"
processed_query, concepts_values = process_query(query_test_Like)
print("Processed Query:", processed_query)
print("Concepts and Values:", concepts_values)

Processed Query: SELECT * FROM films WHERE titre LIKE '%<titre>' ORDER BY annee DESC;
Concepts and Values: [{'concept': 'titre', 'value': 'Promise'}]


In [517]:
lexicon = build_lexicon(BASE_FILMS_URL)
query = "Veuillez me montrer le titre des films où Meryl Streep joue et Hugh Jackman joue."

processed_query, concepts = process_query(query)
print("Processed Query:", processed_query)
print("Extracted Concepts:", concepts)

Processed Query: Veuillez me montrer le titre des films où <actor> joue et <actor> joue.
Extracted Concepts: [{'concept': 'actor', 'value': 'Hugh Jackman'}, {'concept': 'actor', 'value': 'Meryl Streep'}]


In [518]:
import json
import re

def extract_labels_from_sql(sql):
    """
    Extracts labels (parts of the SQL query) as follows:
    - select_label: the string between SELECT and FROM.
    - where_clause: the string between WHERE and ;
    Returns:
    select_label: string with the SELECT portion.
    where_clause: string with the WHERE portion.
    """
    select_label = None
    where_clause = None
    # Get text after SELECT and before FROM
    select_match = re.search(r"(?i)SELECT\s+(.*?)\s+FROM", sql)
    if select_match:
        select_label = select_match.group(1).strip()
    # Get text after WHERE and before ;
    where_match = re.search(r"(?i)WHERE\s+(.*?)\s*;", sql)
    if where_match:
        where_clause = where_match.group(1).strip()
    return select_label, where_clause
#test:
sql = "SELECT * FROM films WHERE titre LIKE 'Project%' ORDER BY annee DESC;"
select_label, where_clause = extract_labels_from_sql(sql)
print("Select Label:", select_label)
print("Where Clause:", where_clause)

Select Label: *
Where Clause: titre LIKE 'Project%' ORDER BY annee DESC


In [519]:
def get_distinct_intentions(json_filename):
    """
    Reads the JSON file with SQL queries and returns a dictionary with
    two keys:
    'select' : containing a list of distinct strings from the SELECT part,
    'where' : containing a list of distinct strings from the WHERE clause.
    """
    lexicon = build_lexicon('base_films_500.csv')
    select_intentions = set()
    where_intentions = set()
    # Read the JSON file
    # Read the JSON file
    with open(json_filename, 'r', encoding='utf-8') as f:
        data = json.load(f)

        # Iterate over each query in the JSON corpus
        for query in data:
            sql = query.get("sql", "")
            sql = process_query(sql)[0]  # Process the SQL query with the lexicon

            select_part, where_part = extract_labels_from_sql(sql)
            if select_part:
                select_intentions.add(select_part)
            if where_part:
                where_intentions.add(where_part)

    return {
        "select": list(select_intentions),
        "where": list(where_intentions)
    }

In [520]:
filename = "queries_french_para.json"
labels = get_distinct_intentions(filename)
print("Labels SELECT:", labels["select"])
print("Labels WHERE:", labels["where"])

Labels SELECT: ['*', 'realisateur', 'acteur1, acteur2, acteur3', 'annee', 'titre']
Labels WHERE: ["realisateur = '<realisateur>'", "titre = '<titre>'", "(acteur1 = '<actor>' OR acteur2 = '<actor>' OR acteur3 = '<actor>') AND genre = '<genre>'", 'annee BETWEEN <annee> AND <annee>', "titre LIKE '<titre>%' ORDER BY annee DESC", "titre LIKE '%<titre>' ORDER BY annee DESC", "((acteur1 = '<actor>' OR acteur2 = '<actor>' OR acteur3 = '<actor>')) AND ((acteur1 = '<actor>' OR acteur2 = '<actor>' OR acteur3 = '<actor>'))", "genre = '<genre>'", 'annee = <annee>']


In [521]:
# %pip install numpy pandas scikit-learn

In [522]:
import numpy as np
from sklearn . model_selection import train_test_split , GridSearchCV
from sklearn . feature_extraction . text import CountVectorizer , TfidfVectorizer
from sklearn . linear_model import Perceptron
from sklearn . pipeline import Pipeline

On commence par le Classifieur du selecteur

In [523]:
def produce_dataset(json_filename):
    """
    Reads the JSON file and returns two lists:
    - X: a list of Natural Language French queries.
    - y: a list of select labels (intents) corresponding to the SQL queries.
    """
    X = []
    y = []
    z = []
    # Read the JSON file
    with open(json_filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
        # Iterate over each query in the JSON corpus
        for query in data:
            sql = query.get("sql", "")
            sql = process_query(sql)[0]
            select_part, where_part = extract_labels_from_sql(sql)
            X.append(process_query(query["english"])[0])
            X.append(process_query(query["french"]["query_french"])[0])
            for x in query["french"]["paraphrase_french"]:
                X.append(process_query(x)[0])
            if where_part:
                z.append(where_part)
                z.append(where_part)
                for x in query["french"]["paraphrase_french"]:
                    z.append(where_part)
            else:
                z.append("")
                z.append("")
                for x in query["french"]["paraphrase_french"]:
                    z.append("")
            if select_part:
                y.append(select_part)
                y.append(select_part)
                for x in query["french"]["paraphrase_french"]:
                    y.append(select_part)
            else:
                y.append("")
                y.append("")
                for x in query["french"]["paraphrase_french"]:
                    y.append("")

    return X, y, z

X, y, z = produce_dataset(filename)


In [524]:
print([query for query in X if "thriller" in query.lower()])
print("X:", X[:10])
print("y:", y[:10])
print("z:", z[:10])

[]
X: ['Please show me the title of the films where <actor> is playing and <actor> is playing.', 'Veuillez me montrer le titre des films où <actor> joue et <actor> joue.', "Montrez-moi, s'il vous plaît, les films dans lesquels <actor> et <actor> apparaissent.", 'Pourriez-vous me donner les titres des films dans lesquels <actor> et <actor> se produisent ?', 'Je souhaiterais connaître les films où <actor> et <actor> ont un rôle.', 'Quels sont les titres des films avec <actor> et <actor> ?', 'Pouvez-vous me dire quels films mettent en vedette <actor> et <actor> ?', 'Please show me the title of the films where <actor> is playing and <actor> is playing.', 'Pouvez-vous me montrer le titre des films dans lesquels <actor> et <actor> jouent ?', 'Pourriez-vous me dire quels films mettent en scène <actor> et <actor> ?']
y: ['titre', 'titre', 'titre', 'titre', 'titre', 'titre', 'titre', 'titre', 'titre', 'titre']
z: ["((acteur1 = '<actor>' OR acteur2 = '<actor>' OR acteur3 = '<actor>')) AND ((acte

In [525]:
X_train , X_test , y_train , y_test = train_test_split (
X , y , test_size =0.2 , random_state =42
)

# Pipeline avec CountVectorizer et Perceptron
pipeline_count = Pipeline([
    ( " vectorizer " , CountVectorizer () ) ,
    ( " perceptron " , Perceptron () )
])
# Entrainement
pipeline_count.fit( X_train , y_train )
# Evaluation
accuracy_count = pipeline_count . score ( X_test , y_test )
print(f" Accuracy avec CountVectorizer : { accuracy_count * 100:.2f} % " )

 Accuracy avec CountVectorizer : 99.94 % 


In [526]:
predictions = pipeline_count.predict( X_test )
for phrase , pred , vrai_label in zip ( X_test , predictions , y_test ) :
    if pred != vrai_label:
        print ( " ------------" )
        print (f" Phrase : { phrase } ")
        print (f" Prediction : { pred }")
        print (f" Label reel : { vrai_label }")
        print ( " ------------" )

 ------------
 Phrase : Montre-moi les interprètes des films de l'année <annee>, s'il te plaît. 
 Prediction : *
 Label reel : acteur1, acteur2, acteur3
 ------------


In [527]:
# Pipeline avec TfidfVectorizer et Perceptron
pipeline_tfidf = Pipeline ([
    ( " vectorizer " , TfidfVectorizer () ) ,
    ( " perceptron " , Perceptron () )
])
pipeline_tfidf . fit ( X_train , y_train )
accuracy_tfidf = pipeline_tfidf . score ( X_test , y_test )
print(f" Accuracy avec TfidfVectorizer : { accuracy_tfidf *100:.2f} % ")

 Accuracy avec TfidfVectorizer : 100.00 % 


In [528]:
vectorizer_params = [
    ( " Count Unigram " , CountVectorizer ( ngram_range =(1 ,1) )),
    ( " Count Bigram " , CountVectorizer ( ngram_range =(2 ,2) )),
    ( " TF - IDF Unigram " , TfidfVectorizer ( ngram_range =(1 ,1) )),
    ( " TF - IDF Uni + Bigram " , TfidfVectorizer ( ngram_range =(1 ,2) )),
]
for name , vec in vectorizer_params :
    pipeline_auto = Pipeline ([
    ( " vectorizer " , vec ) ,
    ( " perceptron " , Perceptron () )
    ])
    pipeline_auto.fit( X_train , y_train )
    score = pipeline_auto.score ( X_test , y_test )
    print (f" { name } - Accuracy : { score } ")

  Count Unigram  - Accuracy : 0.9993746091307066 
  Count Bigram  - Accuracy : 0.9993746091307066 
  TF - IDF Unigram  - Accuracy : 1.0 
  TF - IDF Uni + Bigram  - Accuracy : 0.9993746091307066 


Tout les modèles produisent des résultats parfait
Cela est probablement du à la simplicité du dataset

### Modèle intent pour WHERE

In [529]:
X_train , X_test , z_train , z_test = train_test_split (
X , z , test_size =0.2 , random_state =42
)

# Pipeline avec CountVectorizer et Perceptron
where_pipeline_count = Pipeline([
    ( " vectorizer " , CountVectorizer () ) ,
    ( " perceptron " , Perceptron () )
])
# Entrainement
where_pipeline_count.fit( X_train , z_train )
# Evaluation
accuracy_count = where_pipeline_count . score ( X_test , z_test )
print(f" Accuracy avec CountVectorizer : { accuracy_count * 100:.2f} % " )

 Accuracy avec CountVectorizer : 99.19 % 


In [530]:
predictions = where_pipeline_count.predict( X_test )
for phrase , pred , vrai_label in zip ( X_test , predictions , z_test ) :
    if pred != vrai_label:
        print (f" Query : { phrase } ")
        print (f" Prediction : { pred }")
        print (f" Label reel : { vrai_label }")
        print ( " ------------" )

 Query : Je souhaiterais obtenir des informations sur tous les films comportant 'Infinite' dans le titre, classés par année, du plus récent au plus ancien. 
 Prediction : titre LIKE '<titre>%' ORDER BY annee DESC
 Label reel : titre LIKE '%<titre>' ORDER BY annee DESC
 ------------
 Query : Pouvez-vous afficher toutes les informations sur les films avec le mot Broken dans le titre, en les classant par année décroissante ? 
 Prediction : titre LIKE '<titre>%' ORDER BY annee DESC
 Label reel : titre LIKE '%<titre>' ORDER BY annee DESC
 ------------
 Query : Veuillez me montrer toutes les informations sur les films dont le titre contient Secret, triés par année par ordre décroissant. 
 Prediction : titre LIKE '%<titre>' ORDER BY annee DESC
 Label reel : titre LIKE '<titre>%' ORDER BY annee DESC
 ------------
 Query : Veuillez me montrer toutes les informations des films dont le titre contient Broken, triés par année par ordre décroissant. 
 Prediction : titre LIKE '%<titre>' ORDER BY anne

In [531]:
# Pipeline avec TfidfVectorizer et Perceptron
where_pipeline_tfidf = Pipeline ([
    ( " vectorizer " , TfidfVectorizer () ) ,
    ( " perceptron " , Perceptron () )
])
where_pipeline_tfidf.fit( X_train , z_train )
accuracy_tfidf = where_pipeline_tfidf . score ( X_test , z_test )
print(f" Accuracy avec TfidfVectorizer : { accuracy_tfidf *100:.2f} % ")

 Accuracy avec TfidfVectorizer : 99.12 % 


In [532]:
predictions = where_pipeline_tfidf.predict( X_test )
for phrase , pred , vrai_label in zip ( X_test , predictions , z_test ) :
    if pred != vrai_label:
        print (f" Query : { phrase } ")
        print (f" Prediction : { pred }")
        print (f" Label reel : { vrai_label }")
        print ( " ------------" )

 Query : Montre-moi les interprètes des films de l'année <annee>, s'il te plaît. 
 Prediction : annee BETWEEN <annee> AND <annee>
 Label reel : annee = <annee>
 ------------
 Query : Je souhaiterais obtenir des informations sur tous les films comportant 'Infinite' dans le titre, classés par année, du plus récent au plus ancien. 
 Prediction : realisateur = '<realisateur>'
 Label reel : titre LIKE '%<titre>' ORDER BY annee DESC
 ------------
 Query : Aidez-moi à trouver toutes les informations sur les films ayant 'Broken' dans le titre, triés par date, de la plus récente à la plus ancienne. 
 Prediction : titre LIKE '%<titre>' ORDER BY annee DESC
 Label reel : titre LIKE '<titre>%' ORDER BY annee DESC
 ------------
 Query : Pouvez-vous afficher toutes les informations sur les films avec le mot Broken dans le titre, en les classant par année décroissante ? 
 Prediction : titre LIKE '<titre>%' ORDER BY annee DESC
 Label reel : titre LIKE '%<titre>' ORDER BY annee DESC
 ------------
 Query

In [533]:
vectorizer_params = [
    ( " Count Unigram " , CountVectorizer ( ngram_range =(1 ,1) )),
    ( " Count Bigram " , CountVectorizer ( ngram_range =(2 ,2) )),
    ( " TF - IDF Unigram " , TfidfVectorizer ( ngram_range =(1 ,1) )),
    ( " TF - IDF Uni + Bigram " , TfidfVectorizer ( ngram_range =(1 ,2) )),
]
for name , vec in vectorizer_params :
    where_pipeline_auto = Pipeline ([
    ( " vectorizer " , vec ) ,
    ( " perceptron " , Perceptron () )
    ])
    where_pipeline_auto.fit( X_train , y_train )
    score = where_pipeline_auto.score( X_test , y_test )
    print (f" { name } - Accuracy : { score *100:.2f} % ")

  Count Unigram  - Accuracy : 99.94 % 
  Count Bigram  - Accuracy : 99.94 % 
  TF - IDF Unigram  - Accuracy : 100.00 % 
  TF - IDF Uni + Bigram  - Accuracy : 99.94 % 


Memes Résultats Pour where intent

# Partie 02

## Production de requetes


In [534]:
SELECT_MODEL = pipeline_tfidf
WHERE_MODEL = where_pipeline_tfidf

In [535]:
import re

def predict_sql_query(query, select_pipeline, where_pipeline):
    processed_query, concepts_values = process_query(query)

    # Predict the SQL query
    predicted_selector = select_pipeline.predict([processed_query])[0]
    predicted_where = where_pipeline.predict([processed_query])[0]
    predicted_sql = f"SELECT {predicted_selector} FROM films WHERE {predicted_where};"

    # Replace the placeholders with the actual values
    for concept in concepts_values:
        placeholder = '<' + concept['concept'] + '>'
        if placeholder == '<actor>':
                predicted_sql = predicted_sql.replace(placeholder, concept['value'], 3)
        else:
            predicted_sql = predicted_sql.replace(placeholder, concept['value'], 1)

    return predicted_sql



In [536]:
# Example usage
query = "Donne les films by Steven Spielberg"
predicted_sql = predict_sql_query(query, SELECT_MODEL, WHERE_MODEL)
print("Predicted SQL Query:\n", predicted_sql)

Predicted SQL Query:
 SELECT titre FROM films WHERE realisateur = 'Steven Spielberg';


In [537]:
import pandas as pd
import re

def select_from_csv(csv_file, natural_query):

    sql_query = predict_sql_query(natural_query, SELECT_MODEL, WHERE_MODEL)
    # print("SQL Query:\n", sql_query, "\n\n")
    
    df = pd.read_csv(csv_file)

    # Normalize the SQL query: remove trailing semicolon and extra whitespace.
    sql_query = sql_query.strip().rstrip(";")
    
    # -------------------------------
    # Parse the SELECT clause
    # -------------------------------
    # Matches: SELECT [columns] FROM [table]
    select_match = re.match(r"SELECT\s+(.*?)\s+FROM\s+\w+", sql_query, re.IGNORECASE)
    if not select_match:
        raise ValueError("Invalid SQL query: SELECT clause not found")
    
    columns_str = select_match.group(1).strip()
    if columns_str == "*" or not columns_str:
        selected_columns = list(df.columns)
    else:
        selected_columns = [col.strip() for col in columns_str.split(",")]

    # -------------------------------
    # Parse the WHERE clause, if present.
    # -------------------------------
    where_match = re.search(r"\bWHERE\s+(.*)", sql_query, re.IGNORECASE)
    if where_match:
        where_clause = where_match.group(1).strip()

        # We'll split conditions connected with AND (assuming only AND, not OR).
        conditions = re.split(r'\s+AND\s+', where_clause, flags=re.IGNORECASE)
        
        # For each condition, check if it uses SQL's LIKE operator.
        # We'll apply string filtering for LIKE conditions and collect other conditions.
        remaining_conditions = []
        like_regex = re.compile(r"(\w+)\s+LIKE\s+'(.*?)'", re.IGNORECASE)
        
        for cond in conditions:
            m_like = like_regex.search(cond)
            if m_like:
                column, pattern = m_like.groups()
                # Convert SQL wildcards: '%' to '.*', '_' to '.' for regex.
                regex_pattern = pattern.replace('%', '.*').replace('_', '.')
                # Filter using the pandas Series string method.
                try:
                    df = df[df[column].astype(str).str.contains(regex_pattern, case=False, na=False)]
                except Exception as e:
                    raise ValueError(
                        f"Error filtering column '{column}' with pattern '{pattern}': {e}"
                    )
            else:
                remaining_conditions.append(cond)
                
        # If there are remaining (non-LIKE) conditions, process them via df.query().
        if remaining_conditions:
            # Join conditions with AND.
            remaining_expr = " and ".join(remaining_conditions)
            # Convert SQL equality (=) to Python equality (==) for pandas query.
            # This replacement handles patterns like: column = 'value'
            remaining_expr = re.sub(r"(\w+)\s*=\s*('.*?')", r"\1 == \2", remaining_expr)
            # Convert SQL syntax to Python-compatible for df.query
            remaining_expr = re.sub(r"\bOR\b", "or", remaining_expr, flags=re.IGNORECASE)
            remaining_expr = re.sub(r"\bAND\b", "and", remaining_expr, flags=re.IGNORECASE)
            try:

                df = df.query(remaining_expr)
            except Exception as e:
                raise ValueError(f"Invalid WHERE clause conditions: {e}")

    # -------------------------------
    # Return only the selected columns.
    # -------------------------------
    try:
        return df[selected_columns]
    except KeyError as e:
        raise ValueError(f"Invalid column name in SELECT clause: {e}")


In [538]:
# %pip install jinja2

In [539]:
# from jinja2 import Template
# Example usage
natural_query = "Les titres des films de Leonardo DiCaprio et Brad Pitt"
result_df = select_from_csv(BASE_FILMS_URL, natural_query)
result_df.head()
# result_df.to_latex(index=False)

,titre
236,The Silent Dream Game
277,The Silent Echo Legacy
359,The Frozen Mission Saga


## 3 Evaluation Quantitative

In [540]:
filename_eval = "queries_french_para_eval.json"

In [541]:
import json

def compute_concepts_similarity(json_file_path):
    """
    Reads a JSON file containing queries, processes each record using process_query
    on both the 'sql' field and the 'french' -> 'query_french' field, and computes
    similarity scores between the extracted concepts.

    For each record, the function returns a dictionary containing:
        - The Jaccard similarity score,
        - A Boolean indicating if the extracted concept sets are exactly equal,
        - Precision, Recall, and F1 score (assuming SQL extraction is the reference),
        - Detailed error analysis (false positives and false negatives).
    
    It also returns the average similarity metrics, overall exact match ratio,
    and error analysis counts across all records.
    
    :param json_file_path: Path to the JSON file.
    :return: A dictionary with per-record similarity details and overall averages.
    """
    
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    similarity_results = []
    
    # Initialize accumulators for averages.
    total_jaccard = 0.0
    total_precision = 0.0
    total_recall = 0.0
    total_f1 = 0.0
    n_records = len(data)
    
    # For error analysis accumulators.
    total_false_positives = 0
    total_false_negatives = 0
    exact_match_count = 0
    
    for record in data:
        # Process the SQL query (reference)
        sql_query = record.get('sql', '')
        _, concepts_sql = process_query(sql_query)
        
        # Process the French query
        french_data = record.get('french', {})
        query_french = french_data.get('query_french', '')
        _, concepts_french = process_query(query_french)
        
        # Convert lists of dictionaries to sets of tuples (concept, value)
        set_sql = set((d['concept'], d['value']) for d in concepts_sql)
        set_french = set((d['concept'], d['value']) for d in concepts_french)
        
        # Compute Jaccard similarity: intersection / union
        union = set_sql.union(set_french)
        intersection = set_sql.intersection(set_french)
        jaccard_sim = 1.0 if not union else len(intersection) / len(union)
        
        # Exact match check
        exact_match = set_sql == set_french
        if exact_match:
            exact_match_count += 1
        
        # Define true positives (TP), false positives (FP), and false negatives (FN)
        tp = len(intersection)
        fp = len(set_french - set_sql)
        fn = len(set_sql - set_french)
        
        total_false_positives += fp
        total_false_negatives += fn
        
        # Calculate precision, recall, and F1 score
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
        
        total_jaccard += jaccard_sim
        total_precision += precision
        total_recall += recall
        total_f1 += f1_score
        
        # Detailed error analysis: list the false positives and false negatives
        false_positives = list(set_french - set_sql)
        false_negatives = list(set_sql - set_french)
        
        similarity_results.append({
            'record': record,
            'sql_concepts': set_sql,
            'french_concepts': set_french,
            'jaccard_similarity': jaccard_sim,
            'exact_match': exact_match,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'false_positives': false_positives,
            'false_negatives': false_negatives
        })
    
    # Compute overall average metrics
    average_jaccard = total_jaccard / n_records if n_records > 0 else None
    average_precision = total_precision / n_records if n_records > 0 else None
    average_recall = total_recall / n_records if n_records > 0 else None
    average_f1 = total_f1 / n_records if n_records > 0 else None
    overall_exact_match_ratio = exact_match_count / n_records if n_records > 0 else None
    
    overall_results = {
        "per_record_results": similarity_results,
        "average_jaccard_similarity": average_jaccard,
        "average_precision": average_precision,
        "average_recall": average_recall,
        "average_f1_score": average_f1,
        "overall_exact_match_ratio": overall_exact_match_ratio,
        "total_false_positives": total_false_positives,
        "total_false_negatives": total_false_negatives
    }
    
    return overall_results


In [542]:
detailed_results = compute_concepts_similarity(filename_eval)
print("Overall Exact Match Ratio:", detailed_results["overall_exact_match_ratio"])
print("Average Jaccard Similarity:", detailed_results["average_jaccard_similarity"])
print("Average Precision:", detailed_results["average_precision"])
print("Average Recall:", detailed_results["average_recall"])
print("Average F1 Score:", detailed_results["average_f1_score"])
print("Total False Positives:", detailed_results["total_false_positives"])
print("Total False Negatives:", detailed_results["total_false_negatives"])

Overall Exact Match Ratio: 0.9831410825199645
Average Jaccard Similarity: 0.9831410825199645
Average Precision: 0.9813664596273292
Average Recall: 0.9813664596273292
Average F1 Score: 0.9813664596273292
Total False Positives: 0
Total False Negatives: 19


In [543]:
filename_eval = "queries_french_para_eval.json"

In [544]:
X_eval, y_eval, z_eval = produce_dataset(filename_eval)
print("X_eval:", X_eval[:10])
print("y_eval:", y_eval[:10])   
print("z_eval:", z_eval[:10])

X_eval: ['Please show me the actors of the films where the year is <annee>.', "Veuillez me montrer les acteurs des films où l'année est <annee>.", "Pouvez-vous me donner la liste des acteurs des films de l'année <annee> ?", "Montrez-moi les acteurs des films sortis en <annee>, s'il vous plaît.", "Je voudrais voir les acteurs des films dont l'année est <annee>.", "Est-ce possible d'avoir les noms des acteurs pour les films de l'année <annee> ?", 'Pourrais-je connaître les acteurs des films sortis en <annee> ?', 'Please show me all the information of the films where the year is between <annee> and <annee>.', "Veuillez me montrer toutes les informations des films où l'année est comprise entre <annee> et <annee>.", 'Montrez-moi toutes les infos sur les films sortis entre <annee> et <annee>.']
y_eval: ['acteur1, acteur2, acteur3', 'acteur1, acteur2, acteur3', 'acteur1, acteur2, acteur3', 'acteur1, acteur2, acteur3', 'acteur1, acteur2, acteur3', 'acteur1, acteur2, acteur3', 'acteur1, acteur2

In [545]:

predictions = pipeline_count.predict( X_eval )
# get the accuracy of the model
accuracy_count = pipeline_count . score ( X_eval , y_eval )
print(f" Accuracy avec CountVectorizer : { accuracy_count * 100:.2f} % " )

for phrase , pred , vrai_label in zip ( X_eval , predictions , y_eval ) :
    if pred != vrai_label:
        print ( " ------------" )
        print (f" Phrase : { phrase } ")
        print (f" Prediction : { pred }")
        print (f" Label reel : { vrai_label }")
        print ( " ------------" )

 Accuracy avec CountVectorizer : 99.63 % 
 ------------
 Phrase : Veuillez me montrer le titre des films dont le réalisateur est les frères Coen. 
 Prediction : realisateur
 Label reel : titre
 ------------
 ------------
 Phrase : J'aimerais connaître le titre des films réalisés par les frères Coen. 
 Prediction : realisateur
 Label reel : titre
 ------------
 ------------
 Phrase : Serait-il possible d'obtenir la liste des films des frères Coen ? 
 Prediction : acteur1, acteur2, acteur3
 Label reel : titre
 ------------
 ------------
 Phrase : Please show me the director and the total number of films, grouped by realisateur sorted in descending order. 
 Prediction : titre
 Label reel : realisateur, COUNT(*) AS total
 ------------
 ------------
 Phrase : Veuillez me montrer le directeur et le nombre total de films, groupés par réalisateur, triés par ordre décroissant. 
 Prediction : *
 Label reel : realisateur, COUNT(*) AS total
 ------------
 ------------
 Phrase : Pourriez-vous me mo

In [546]:

predictions = pipeline_tfidf.predict( X_eval )
# get the accuracy of the model
accuracy_count = pipeline_tfidf . score ( X_eval , y_eval )
print(f" Accuracy avec TF-IDF : { accuracy_count * 100:.2f} % " )

for phrase , pred , vrai_label in zip ( X_eval , predictions , y_eval ) :
    if pred != vrai_label:
        print ( " ------------" )
        print (f" Phrase : { phrase } ")
        print (f" Prediction : { pred }")
        print (f" Label reel : { vrai_label }")
        print ( " ------------" )

 Accuracy avec TF-IDF : 99.66 % 
 ------------
 Phrase : Veuillez me montrer le titre des films dont le réalisateur est les frères Coen. 
 Prediction : realisateur
 Label reel : titre
 ------------
 ------------
 Phrase : Pourriez-vous me donner les titres des films des frères Coen ? 
 Prediction : annee
 Label reel : titre
 ------------
 ------------
 Phrase : Pouvez-vous me dire quels films ont été réalisés par les frères Coen ? 
 Prediction : acteur1, acteur2, acteur3
 Label reel : titre
 ------------
 ------------
 Phrase : Please show me the director and the total number of films, grouped by realisateur sorted in descending order. 
 Prediction : *
 Label reel : realisateur, COUNT(*) AS total
 ------------
 ------------
 Phrase : Veuillez me montrer le directeur et le nombre total de films, groupés par réalisateur, triés par ordre décroissant. 
 Prediction : *
 Label reel : realisateur, COUNT(*) AS total
 ------------
 ------------
 Phrase : Pourriez-vous me montrer qui est le réal

In [547]:
#count:
predictions = where_pipeline_count.predict( X_eval )
# get the accuracy of the model
where_accuracy_count = where_pipeline_count . score ( X_eval , z_eval )
print(f" Accuracy avec countVectorizer : { where_accuracy_count *100:.2f} % " )

for phrase , pred , vrai_label in zip ( X_eval , predictions , z_eval ) :
    if pred != vrai_label:
        print ( " ------------" )
        print (f" Phrase : { phrase } ")
        print (f" Prediction : { pred }")
        print (f" Label reel : { vrai_label }")
        print ( " ------------" )


 Accuracy avec countVectorizer : 98.90 % 
 ------------
 Phrase : Veuillez me montrer le titre des films dont le réalisateur est les frères Coen. 
 Prediction : titre = '<titre>'
 Label reel : realisateur = '<realisateur>'
 ------------
 ------------
 Phrase : Serait-il possible d'obtenir la liste des films des frères Coen ? 
 Prediction : titre = '<titre>'
 Label reel : realisateur = '<realisateur>'
 ------------
 ------------
 Phrase : Please show me all the information of the films where the title contains Last, sorted by year in descending order. 
 Prediction : titre LIKE '%<titre>' ORDER BY annee DESC
 Label reel : titre LIKE '<titre>%' ORDER BY annee DESC
 ------------
 ------------
 Phrase : Veuillez montrer toutes les informations des films dont le titre contient Last, triés par année par ordre décroissant. 
 Prediction : titre LIKE '%<titre>' ORDER BY annee DESC
 Label reel : titre LIKE '<titre>%' ORDER BY annee DESC
 ------------
 ------------
 Phrase : Montrez-moi toutes les

In [548]:
#tfidf:
predictions = where_pipeline_tfidf.predict( X_eval )
# get the accuracy of the model
where_accuracy_tfidf = where_pipeline_tfidf . score ( X_eval , z_eval )
print(f" Accuracy avec TfidfVectorizer : { where_accuracy_tfidf *100:.2f} % " )

for phrase , pred , vrai_label in zip ( X_eval , predictions , z_eval ) :
    if pred != vrai_label:
        print ( " ------------" )
        print (f" Phrase : { phrase } ")
        print (f" Prediction : { pred }")
        print (f" Label reel : { vrai_label }")
        print ( " ------------" )


 Accuracy avec TfidfVectorizer : 98.90 % 
 ------------
 Phrase : Veuillez me montrer le titre des films dont le réalisateur est les frères Coen. 
 Prediction : titre = '<titre>'
 Label reel : realisateur = '<realisateur>'
 ------------
 ------------
 Phrase : Pourriez-vous me donner les titres des films des frères Coen ? 
 Prediction : (acteur1 = '<actor>' OR acteur2 = '<actor>' OR acteur3 = '<actor>') AND genre = '<genre>'
 Label reel : realisateur = '<realisateur>'
 ------------
 ------------
 Phrase : J'aimerais connaître le titre des films réalisés par les frères Coen. 
 Prediction : titre = '<titre>'
 Label reel : realisateur = '<realisateur>'
 ------------
 ------------
 Phrase : Serait-il possible d'obtenir la liste des films des frères Coen ? 
 Prediction : titre = '<titre>'
 Label reel : realisateur = '<realisateur>'
 ------------
 ------------
 Phrase : Pouvez-vous me dire quels films ont été réalisés par les frères Coen ? 
 Prediction : annee = <annee>
 Label reel : realisa

### 3.3 Testing values

In [549]:
filename_eval = "queries_french_para_eval_values.json"

In [550]:
with open(filename_eval, "r", encoding="utf-8") as file:
    data = json.load(file)

correct_predictions = 0
total_predictions = 0
error_details = []  # To store records of errors for further error analysis

for record in data:
    query_text = record.get("query", "")
    expected_value = record.get("value", "").lower().strip()
    
    try:
        df_predicted = select_from_csv(BASE_FILMS_URL, query_text)
    except Exception as e:
        error_details.append({
            "query": query_text,
            "expected": expected_value,
            "predicted": None,
            "error": str(e)
        })
        total_predictions += 1
        continue

    if df_predicted.empty:
        predicted_value = None
    else:
        predicted_value = str(df_predicted.iloc[0, 0]).lower().strip()
    
    total_predictions += 1

    if predicted_value == expected_value:
        correct_predictions += 1
    else:
        error_details.append({
            "query": query_text,
            "expected": expected_value,
            "predicted": predicted_value,
            "error": None
        })

accuracy_score = correct_predictions / total_predictions if total_predictions > 0 else 0

print("Accuracy Score:", accuracy_score)
print("Total Predictions:", total_predictions)
print("Correct Predictions:", correct_predictions)
print("\nErreurs (Pour analyse):")
for err in error_details:
    print(err)

Accuracy Score: 0.9982300884955753
Total Predictions: 1695
Correct Predictions: 1692

Erreurs (Pour analyse):
{'query': "Pourriez-vous me dire qui s'occupe de la direction des films The Broken Dream Code ?", 'expected': 'stanley kubrick', 'predicted': '1980', 'error': None}
{'query': "Quelles sont les années des films intitulés The Lost Echo Files, s'il te plaît ?", 'expected': '2012', 'predicted': 'christopher nolan', 'error': None}
{'query': 'Pourriez-vous me montrer les années des films intitulés The Hidden Memory Legacy ?', 'expected': '1984', 'predicted': 'wes anderson', 'error': None}


In [551]:
query = "Pourriez-vous me montrer les années des films intitulés The Hidden Memory Legacy ?"
predicted_sql = predict_sql_query(query, SELECT_MODEL, WHERE_MODEL)
print("Predicted SQL Query:\n", predicted_sql)

Predicted SQL Query:
 SELECT realisateur FROM films WHERE titre = 'The Hidden Memory Legacy';


In [552]:
queries = [
    "Quels films ont été réalisés par Christopher Nolan ?",
    "Quel est le réalisateur du film 'The Silent Memory Files' ?",
    "Peux-tu me donner la liste des films sortis en 1994 ?",
    "Quels films appartiennent au genre 'drame' ?",
    "Quels films a réalisé Steven Spielberg ?",
    "Qui sont les acteurs du film 'The Infinite Mission Game' ?",
    "Qui a réalisé le film 'The Last Truth Protocol' ?",
    "Quels films ont un titre contenant 'Shadow' ?",
    "Quel est le genre du film 'The Golden Memory Code' ?",
    "Liste les films où apparaît Leonardo DiCaprio.",
    "Quels films d'action sont sortis entre 2010 et 2012 ?",
    "Peux-tu me montrer les films avec Brad Pitt sortis avant 2000 ?",
    "Donne-moi le titre et le réalisateur des films de 2005.",
    "Quelles comédie sont sortis après 2015 ?",
    "Qui est le réalisateur du film 'The Infinite Secret Project' ?",
    "Quel film de thriller est sorti en 1986 ?",
    "Quels films réunissent Leonardo DiCaprio et Emma Stone ?",
    "Quels films ont été signés par Martin Scorsese ?",
    "Quels films du genre horreur sont sortis en 2020 ?",
    "Liste les films d'action dont le titre commence par 'The'."
]

for query in queries:
    print("Query:\n", query)
    print("Predicted SQL Query:\n", predict_sql_query(query, SELECT_MODEL, WHERE_MODEL))


Query:
 Quels films ont été réalisés par Christopher Nolan ?
Predicted SQL Query:
 SELECT titre FROM films WHERE realisateur = 'Christopher Nolan';
Query:
 Quel est le réalisateur du film 'The Silent Memory Files' ?
Predicted SQL Query:
 SELECT realisateur FROM films WHERE titre = 'The Silent Memory Files';
Query:
 Peux-tu me donner la liste des films sortis en 1994 ?
Predicted SQL Query:
 SELECT acteur1, acteur2, acteur3 FROM films WHERE annee = 1994;
Query:
 Quels films appartiennent au genre 'drame' ?
Predicted SQL Query:
 SELECT titre FROM films WHERE (acteur1 = '<actor>' OR acteur2 = '<actor>' OR acteur3 = '<actor>') AND genre = 'Drame';
Query:
 Quels films a réalisé Steven Spielberg ?
Predicted SQL Query:
 SELECT titre FROM films WHERE realisateur = 'Steven Spielberg';
Query:
 Qui sont les acteurs du film 'The Infinite Mission Game' ?
Predicted SQL Query:
 SELECT acteur1, acteur2, acteur3 FROM films WHERE annee = <annee>;
Query:
 Qui a réalisé le film 'The Last Truth Protocol' ?


In [553]:
query = "Liste les films ou joue Leonardo DiCaprio et dont le genre est 'action'."
predicted_sql = predict_sql_query(query, SELECT_MODEL, WHERE_MODEL)
print("Predicted SQL Query:\n", predicted_sql)

Predicted SQL Query:
 SELECT titre FROM films WHERE (acteur1 = 'Leonardo DiCaprio' OR acteur2 = 'Leonardo DiCaprio' OR acteur3 = 'Leonardo DiCaprio') AND genre = 'Action';


In [554]:
query = "films d'action de 2010"
predicted_sql = predict_sql_query(query, SELECT_MODEL, WHERE_MODEL)
print("Predicted SQL Query:\n", predicted_sql)

Predicted SQL Query:
 SELECT * FROM films WHERE (acteur1 = '<actor>' OR acteur2 = '<actor>' OR acteur3 = '<actor>') AND genre = 'Action';
